In [1]:
import numpy as np
from matplotlib import pyplot as plt
from NeuralNetwork import NeuralNetwork
from utils import get_data, onehot_encoding, grid_search, save_dict_to_file, load_dict_from_file
from Layer import Layer, Input
from functions import accuracy, MSE, MEE
import pandas as pd

In [2]:
best_comb_filename = '/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/output/best_comb_cup2.pkl'
param_grid_filename = '/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/output/param_grid_cup2.pkl'

In [3]:
names = ['id', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 
         'feature_7', 'feature_8', 'feature_9', 'feature_10', 'target_x', 'target_y','target_z']

df = pd.read_csv('/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/ML-CUP23-TR.csv', names=names, comment='#')

targets = ['target_x', 'target_y', 'target_z']
features = list(set(names) - {'id', 'target_x', 'target_y', 'target_z'})

df = df.sample(frac=1)
prova = df[:750]
X_train, y_train = prova[features].to_numpy().T, prova[targets].to_numpy().T
X_test, y_test = df[750:][features].to_numpy().T, df[750:][targets].to_numpy().T

In [4]:
best_comb = load_dict_from_file(best_comb_filename)
param_grid = load_dict_from_file(param_grid_filename)

In [5]:
# Print the best {how_many} hyperparameter sets by their {sorting_key}

how_many = 5 # how many elements to print
sorting_key = 'val_MEE_mean' # value on which the sorting is performed
p_best = []

p_to_sort = [[x, x['results'][sorting_key]] for x in param_grid]
p_best_sorted = sorted(p_to_sort, key=lambda x:x[1])

for p_comb,loss in p_best_sorted[:how_many]:
    p_best.append(p_comb)
    tmp_dict = p_comb.copy()
    results = tmp_dict.pop('results')
    print(f"val_loss = {results['val_loss_mean']:.2e} +- {results['val_loss_std']:.2e}")
    print(f"val_MEE = {results['val_MEE_mean']:.2e} +- {results['val_MEE_std']:.2e}")
    print(f'hyperparameters : {tmp_dict}')
    print(f'results : {results}')
    print('')

val_loss = 9.53e-01 +- 1.13e-01
val_MEE = 1.40e+00 +- 6.19e-02
hyperparameters : {'eta': 0.009, 'lam': 0.1, 'alpha': 1.2, 'epochs': 500, 'n_batch': 150, 'scale_eta_batchsize': None, 'dim_hidden': 25, 'hidden_act_func': 'leaky_relu', 'dim_hidden2': 30, 'hidden_act_func2': 'leaky_relu', 'dim_hidden3': 10, 'hidden_act_func3': 'leaky_relu', 'use_opt': 0, 'loss': 'MSE', 'output_act_func': 'lin', 'elapsed_time': 961.4998867511749}
results : {'train_loss_mean': 0.6172068548431464, 'train_loss_std': 0.06985605292252886, 'val_loss_mean': 0.9533494896487491, 'val_loss_std': 0.11312852581938311, 'train_MEE_mean': 1.1745880551282362, 'train_MEE_std': 0.08077716558125936, 'val_MEE_mean': 1.4037974952022187, 'val_MEE_std': 0.06193356014637131}

val_loss = 9.96e-01 +- 8.57e-02
val_MEE = 1.41e+00 +- 4.75e-02
hyperparameters : {'eta': 0.007, 'lam': 0.1, 'alpha': 0.9, 'epochs': 500, 'n_batch': 150, 'scale_eta_batchsize': None, 'dim_hidden': 25, 'hidden_act_func': 'leaky_relu', 'dim_hidden2': 30, 'hidden

In [6]:
# Permorm retraining on the best {how_many} sets and print loss and MEE graphs.

retraining_epochs = 250
print_best = True # print the best results on all graphs

for i,p_comb in enumerate(p_best):

    best_comb = p_comb.copy()

    best_comb['epochs'] = retraining_epochs

    results = best_comb.pop('results')
    if best_comb['n_batch'] == 'batch':
        best_comb['n_batch'] = X_train.shape[1]
    elapsed_time = best_comb.pop('elapsed_time')
    if best_comb['scale_eta_batchsize'] == 'lin':
        best_comb['eta'] = best_comb['eta'] * best_comb['n_batch']
    if best_comb['scale_eta_batchsize'] == 'sqrt':
        best_comb['eta'] = best_comb['eta'] * np.sqrt(best_comb['n_batch'])
    best_comb.pop('scale_eta_batchsize')

    # ----------------------- MODEL ----------------------------
    
    input_layer = Input(X_train.shape[0])
    hidden_layer = Layer(input_layer, best_comb.pop('dim_hidden'), best_comb.pop('hidden_act_func'))
    o = 2
    while True:
        if f'dim_hidden{o}' in best_comb.keys():
            hidden_layer = Layer(hidden_layer, best_comb.pop(f'dim_hidden{o}'), best_comb.pop(f'hidden_act_func{o}'))
            o += 1
        else: break
    output_layer = Layer(hidden_layer, 3, best_comb.pop('output_act_func'))

    model = NeuralNetwork(input_layer, output_layer, best_comb.pop('loss'), metrics=[MEE])
    history = model.retrain(X_train, y_train.reshape((3,X_train.shape[1])), test_data = [X_test,y_test.reshape((3,X_test.shape[1]))], **best_comb)

    # ----------------------------------------------------------

    if i == 0:
        best_train_loss, best_test_loss = history['train_loss'], history['test_loss']
        best_train_MEE, best_test_MEE = history['train_MEE'], history['test_MEE']

    plt.figure(i, figsize=(30,10))
    
    plt.subplot(1,2,1)
    plt.plot(history['train_loss'],label='train_loss')
    plt.plot(history['test_loss'], label='test_loss')
    if (i > 0) and (print_best == True):
        plt.plot(best_train_loss,'--',label='best_train_loss')
        plt.plot(best_test_loss, '--', label='best_test_loss')
    plt.yscale('log')
    plt.xlabel('Epochs', size=15)
    plt.ylabel('Loss functions', size=15)
    plt.title('train_loss vs test_loss', size=18)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.grid()
    plt.legend(fontsize=15)

    plt.subplot(1,2,2)
    plt.plot(history['train_MEE'],label='train_MEE')
    plt.plot(history['test_MEE'], label='test_MEE')
    if (i > 0) and (print_best == True):
        plt.plot(best_train_MEE,'--',label='best_train_MEE')
        plt.plot(best_test_MEE, '--', label='best_test_MEE')
    plt.xlabel('Epochs', size=15)
    plt.ylabel('MEE', size=15)
    plt.title('train_MEE vs test_MEE', size=18)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.grid()
    plt.legend(fontsize=15)

    plt.show()

Epoch 0: train_loss: 7.665e+02, test_loss: 7.473e+02; train_MEE: 4.282e+01, test_MEE: 4.266e+01  lr : 0.009
Epoch 1: train_loss: 7.402e+02, test_loss: 7.210e+02; train_MEE: 4.214e+01, test_MEE: 4.195e+01  lr : 0.009
Epoch 2: train_loss: 6.978e+02, test_loss: 6.749e+02; train_MEE: 4.100e+01, test_MEE: 4.061e+01  lr : 0.009
Epoch 3: train_loss: 6.075e+02, test_loss: 5.805e+02; train_MEE: 3.844e+01, test_MEE: 3.773e+01  lr : 0.009
Epoch 4: train_loss: 4.638e+02, test_loss: 4.235e+02; train_MEE: 3.373e+01, test_MEE: 3.228e+01  lr : 0.009
Epoch 5: train_loss: 2.859e+02, test_loss: 2.562e+02; train_MEE: 2.632e+01, test_MEE: 2.498e+01  lr : 0.009
Epoch 6: train_loss: 1.592e+02, test_loss: 1.459e+02; train_MEE: 2.003e+01, test_MEE: 1.917e+01  lr : 0.009
Epoch 7: train_loss: 1.120e+02, test_loss: 1.065e+02; train_MEE: 1.695e+01, test_MEE: 1.642e+01  lr : 0.009
Epoch 8: train_loss: 1.091e+02, test_loss: 1.031e+02; train_MEE: 1.681e+01, test_MEE: 1.634e+01  lr : 0.009
Epoch 9: train_loss: 8.392e+

KeyboardInterrupt: 

In [ ]:
def function(seed):
    np.random.seed(seed)
    return np.random.uniform(0,10,10)

In [ ]:
print(function(232323))

[5.58526844 2.1846418  6.79919952 7.11004665 7.14118482 0.5398932
 0.13095949 8.0790181  8.17264435 3.85478113]


: 